<a href="https://colab.research.google.com/github/microdinosaur/10kAnalysis/blob/master/anomalyDetectionCloudFunction-TestEnv/Step_In_Testing_Dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [0]:
from datetime import datetime, timedelta
import datetime as dt
import random
import os
from sklearn.metrics import mean_absolute_error
import numpy as np                               # vectors and matrices
import pandas as pd                              # tables and data manipulations
from google.cloud import bigquery
import google.cloud.bigquery as bq
import ast
import pyarrow

# Set Env Variables

In [0]:
sql = """SELECT myDate, source, medium, landingPage, sessions, goalCompletions  FROM `michaelr-workspace.anomalyDetectionRoot.cbtNuggets` WHERE myDate between DATE_SUB(current_date(), INTERVAL 400 DAY) and current_date()""" #@param
id = 'michaelr-workspace' #@param {type: "string"}
toClean = True #@param
colToClean = 'landingPage' #@param {type: "string"}
dateColumn = 'myDate' #@param {type: "string"}
dimColumnOne = 'landingPage' #@param {type: "string"}
dimColumnTwo = 'source' #@param {type: "string"}
metricColumn = 'sessions' #@param {type: "string"}
finalTable = 'BREAK' #@param {type: "string"}
lpCount = 100 #@param {type: "number"}
series_length = 365 #@param {type: "number"}
window = 8 #@param {type: "number"}
focusList = "['www.cbtnuggets.com/',  'www.cbtnuggets.com/learn',  '(not set)',  'www.cbtnuggets.com/login',  'www.cbtnuggets.com/search']" #@param {type: "string"}
focusList = ast.literal_eval(focusList)

# Get tables for testing

In [0]:
keyName = 'michaelROwnerKey.json'
client = bq.Client.from_service_account_json(keyName)
df = client.query(sql, project=id).to_dataframe()


In [0]:
dfbq = df.copy()

# Set Functions Up

## Data Frame Creation

In [0]:
def prepSingleDataFrame(df, clean, cleanCol, dateCol, metric, *dims):
    dfFun = df.copy()
    del df
    if clean == True:
        dfFun[colToClean] = dfFun[colToClean].str.replace('\?.*', '')  # Clean Query Strings from LPs
    dfOut = dfFun.copy()
    del dfFun
    cols = []
    cols.append(dateCol)
    for i in dims:
        cols.append(i)
    cols.append(metric)
    dfOut = dfOut[cols]
    return dfOut

## Filter Tables

In [0]:
def filterDim(priorityPages, df, filterDim):
    dfFiltered = df.loc[df[filterDim].isin(priorityPages)]
    return dfFiltered

## Create now and later tables

In [0]:
def nowAndLater(df, date, met, start, end, startYoY, endYoY, *dims):
    ### Now ###
    dfNow = df[df[date].between(start, end)]  # filter to date
    # create grouping index
    groups = []
    groups.append(date)
    for i in dims:
        groups.append(i)

    dfNow = dfNow.groupby(groups).sum()

    dfNow.reset_index(inplace=True)

    dfNow[date] = pd.to_datetime(dfNow[date], format='%Y-%m-%d')

    dfNowFinal = dfNow.set_index(groups, inplace=False)

    ### Then ###
    dfThen = df[df[date].between(startYoY, endYoY)]  # filter to date
    # create grouping index
    groups = []
    groups.append(date)
    for i in dims:
        groups.append(i)

    dfThen = dfThen.groupby(groups).sum()
    dfThen.reset_index(inplace=True)
    dfNow[date] = pd.to_datetime(dfThen[date], format='%Y-%m-%d')
    dfThenFinal = dfThen.set_index(groups, inplace=False)

    return dfNowFinal, dfThenFinal

## Get Intersections

In [0]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

## Get Bounds

In [0]:
def getBounds(series, window, rollingMean, scale=1, lowerBound=True):

    #print("Get Bounds - Series: ", series)
    #print("Get Bounds - Window: ", window)
    #print("Get Bounds - rollingMean: ", rollingMean)
    if lowerBound == True:
        boundAdjuster = -1
    else:
        boundAdjuster = 1

    mae = mean_absolute_error(series[window:], rollingMean[window:])  # Get MAE

    deviation = np.std(series[window:] - rollingMean[window:])  # Set Deviation

    bound = rollingMean + boundAdjuster * \
    	(mae + scale * deviation)  # Set Lower Bound

    return bound

## Get Anomaly Function

In [0]:
def getAnomalies2(series, window, upperBound, lowerBound, upperBoundYOY=[], lowerBoundYOY=[], lbound=True, isYOY=True):
    try:
        anomalies = pd.DataFrame(
            index=series.index, columns=series.columns)  # Set Anomalies
        #print("Length of Anomalies: ", len(anomalies))
        anomalies[series < lowerBound] = series[series < lowerBound]
        # print("Error Handling: anomalies lower",anomalies)
        anomalies[series > upperBound] = series[series > upperBound]
        # print("Error Handling: anomalies upper",anomalies)
        suspect_lower = anomalies[series < lowerBound].dropna(how='all')
        # print("Error Handling: suspect_lower", suspect_lower,"\n")
        suspect_upper = anomalies[series > upperBound].dropna(how='all')
        # print("Error Handling: suspect_upper", suspect_upper,"\n")

        if isYOY == True:
            #print("Get Anomalies  - isYOY is True")
            anomaliesYOY = pd.DataFrame(
            	index=series.index, columns=series.columns)
            #print("Error Handling: anomaliesYOY",anomaliesYOY)
            #print("Error Handling: series", series)
            anomaliesYOY[series <
                         lowerBoundYOY] = series[series < lowerBoundYOY]
            anomaliesYOY[series > upperBoundYOY] = series[series >
                                                          upperBoundYOY]  # REMOVE UPPER BOUNDS CHECK

            suspect_lowerYOY = anomaliesYOY[series <
                                            lowerBoundYOY].dropna(how='all')
            # print("Error Handling: suspect_lowerYOY", suspect_lowerYOY)
            suspect_upperYOY = anomaliesYOY[series > upperBoundYOY].dropna(
            	how='all')  # REMOVE UPPER BOUNDS CHECK
            suspectYOY = anomaliesYOY.dropna(how='all')

            ## Merge both years
            realAnomaliesLower = pd.merge(
            	suspect_lower, suspect_lowerYOY, right_index=True, left_index=True)
            realAnomaliesUpper = pd.merge(
            	suspect_upper, suspect_upperYOY, right_index=True, left_index=True)

            if lbound == True:
                  return realAnomaliesLower
            if lbound == False:
                  return realAnomaliesUpper

        elif isYOY == False:
            realAnomaliesLower = pd.merge(
            	suspect_lower, suspect_lower, right_index=True, left_index=True)
            realAnomaliesUpper = pd.merge(
            	suspect_upper, suspect_upper, right_index=True, left_index=True)

            if lbound == True:
                return realAnomaliesLower
            if lbound == False:
                return realAnomaliesUpper

    except:
        thisFailed = series['sessions'] = np.nan
        return anomalies

#OG Handler

In [0]:
def handler(dfbq,colToClean, dateColumn,dimColumnOne,dimColumnTwo,metricColumn,toClean=True, lpCount=100, series_length=365, window=8):

    #Generate Query & initial table
    dfCopy = dfbq.copy()
    del dfbq
    print("1. BigQuery Table Collected")

    # Create Initial DataFrame - Cleaning Query Strings if Applicable
    #def prepSingleDataFrame(df, clean, cleanCol, dateCol, metric, *dims):
    df = prepSingleDataFrame(dfCopy, toClean, colToClean,
                             dateColumn, metricColumn, dimColumnOne, dimColumnTwo)
    print("\n2. Data Frame Created")

    # Get Filter list for primary dimension
    #focusList = focus(df, dimColumnOne, metricColumn, n=lpCount)
    #print("\n3. Focus list created of top ", lpCount, dimColumnOne)
    

    # Update table to only include focusList items
    dfFiltered = filterDim(focusList, df, dimColumnOne)
    print("\n4. Filtered data to top ", lpCount, dimColumnOne)
    del df

    # Set Dates
    end_date = pd.to_datetime(
    	(datetime.now() - timedelta(1)).strftime('%Y-%m-%d')).date()
    start_date = pd.to_datetime(
    	(datetime.now() - timedelta(30)).strftime('%Y-%m-%d')).date()
    start_dateYOY = pd.to_datetime(
    	(datetime.now() - timedelta(396)).strftime('%Y-%m-%d')).date()  # test at 396,365
    end_dateYOY = pd.to_datetime(
    	(datetime.now() - timedelta(366)).strftime('%Y-%m-%d')).date()
    print("\n5. Created Data ranges")

    # Create Current Year Table and Previous Year Table
    dfNow, dfThen = nowAndLater(dfFiltered, dateColumn, metricColumn,
                                start_date, end_date, start_dateYOY, end_dateYOY, dimColumnOne)
    print("\n6. Created current and previous year tables")
    del end_date, start_date, start_dateYOY, end_dateYOY

    # Create Columns Item
    cols = dfThen.columns

    # Create Categories
    dfCategories = intersection(dfThen.index.get_level_values(
    	1).unique(), dfNow.index.get_level_values(1).unique())

    # Create Category Counter - Set to 0
    categoryCounter = 0  # IS THIS NECESSARY?=['/]

    # Create Initial Data Frame
    tempFrames = []

    # Error Handling, print my list of categories
    print(dfCategories)
    print('\n6. Begin looping through dimension')

    # Loop Through
    for category in dfCategories:
        print("running function: ", category)

        # Current Year Data
        dfTemp = dfNow.loc(axis=0)[pd.IndexSlice[:, category]].copy()
        dfTempYOY = dfThen.loc(
            axis=0)[pd.IndexSlice[:, category]].copy()   # YOY Data

        # Current Year Data
        dfTemp.reset_index(level=[1], inplace=True)
        # YOY Data
        dfTempYOY.reset_index(level=[1], inplace=True)

        print("\n\n\nAnomalies by " +
              dfTemp.columns[0] + ": " + category + ' \n')
        # Current Year Data
        dfTemp.drop(dfTemp.columns[0], axis=1, inplace=True)
        # YOY Data
        dfTempYOY.drop(dfTempYOY.columns[0], axis=1, inplace=True)
        # Shift dates forward one day
        dfTempYOY.index = dfTempYOY.index.shift(0, freq='D')
        dfTempYOY = dfTempYOY.tail(30)
        dfTemp = dfTemp.asfreq('D').fillna(0)  # FillNA with O
        dfTempYOY = dfTempYOY.asfreq('D').fillna(0)  # FillNA with O

        # Assign Series
        series = dfTemp
        seriesYOY = dfTempYOY
        #print(series)
        #print(seriesYOY)

        if len(series) > window:

            # Set series
            series = series.tail(series_length)

            # Get rolling mean for current year
            rolling_mean = series.rolling(window=window).mean()

            # create Len of Series Test values
            seriesLen = len(series)
            seriesLenYOY = len(seriesYOY)

            #shift index forward 1 year to overlay last year's data over current year's
            seriesYOY.index = seriesYOY.index.shift(365, freq='D')
            seriesYOY = seriesYOY.tail(series_length)

            # Create YOY Rolling Mean
            rolling_meanYOY = seriesYOY.rolling(window=window).mean()

            # Create yoyTest Variable
            if seriesLen == seriesLenYOY:  # Both Series are Equal, run current and YOY
                yoyTest = True
            elif seriesLen > seriesLenYOY:
                yoyTest = False
            elif seriesLen != seriesLenYOY:
                yoyTest = False

            # Create Bounds
            upperBoundTierOne = getBounds(
            	series, window, rolling_mean, scale=1, lowerBound=False)
            lowerBoundTierOne = getBounds(
            	series, window, rolling_mean, scale=1, lowerBound=True)

            upperBoundTierTwo = getBounds(
            	series, window, rolling_mean, scale=2, lowerBound=False)
            lowerBoundTierTwo = getBounds(
            	series, window, rolling_mean, scale=2, lowerBound=True)

            upperBoundTierThree = getBounds(
            	series, window, rolling_mean, scale=3, lowerBound=False)
            lowerBoundTierThree = getBounds(
            	series, window, rolling_mean, scale=3, lowerBound=True)

            # Create YOY Bounds
            if yoyTest == True:  # Create YOY if YOY Is True
                upperBoundYOYTierOne = getBounds(
                    seriesYOY, window, rolling_meanYOY, scale=1, lowerBound=False)
                lowerBoundYOYTierOne = getBounds(
                    seriesYOY, window, rolling_meanYOY, scale=1, lowerBound=True)
                upperBoundYOYTierTwo = getBounds(
                    seriesYOY, window, rolling_meanYOY, scale=2, lowerBound=False)
                lowerBoundYOYTierTwo = getBounds(
                    seriesYOY, window, rolling_meanYOY, scale=2, lowerBound=True)
                upperBoundYOYTierThree = getBounds(
                    seriesYOY, window, rolling_meanYOY, scale=3, lowerBound=False)
                lowerBoundYOYTierThree = getBounds(
                    seriesYOY, window, rolling_meanYOY, scale=3, lowerBound=True)
            else:
                replaceVal = series.copy()
                #print("Copy: ", replaceVal)
                replaceVal[metricColumn] = np.nan
                #print(replaceVal)
                upperBoundYOYTierOne = replaceVal
                lowerBoundYOYTierOne = replaceVal
                upperBoundYOYTierTwo = replaceVal
                lowerBoundYOYTierTwo = replaceVal
                upperBoundYOYTierThree = replaceVal
                lowerBoundYOYTierThree = replaceVal

            #print(series['sessions'])
            #print(upperBoundYOYTierOne)

            # Find Anomalies
            # Tier One
            allAnomaliesUpperTierOne = getAnomalies2(series, window, upperBoundTierOne, lowerBoundTierOne,
                                                     upperBoundYOY=upperBoundYOYTierOne, lowerBoundYOY=lowerBoundYOYTierOne, lbound=False, isYOY=yoyTest)
            allAnomaliesLowerTierOne = getAnomalies2(series, window, upperBoundTierOne, lowerBoundTierOne,
                                                     upperBoundYOY=upperBoundYOYTierOne, lowerBoundYOY=lowerBoundYOYTierOne, lbound=True, isYOY=yoyTest)

            # Tier Two
            allAnomaliesUpperTierTwo = getAnomalies2(series, window, upperBoundTierTwo, lowerBoundTierTwo,
                                                     upperBoundYOY=upperBoundYOYTierTwo, lowerBoundYOY=lowerBoundYOYTierTwo, lbound=False, isYOY=yoyTest)
            allAnomaliesLowerTierTwo = getAnomalies2(series, window, upperBoundTierTwo, lowerBoundTierTwo,
                                                     upperBoundYOY=upperBoundYOYTierTwo, lowerBoundYOY=lowerBoundYOYTierTwo, lbound=True, isYOY=yoyTest)

            # Tier Three
            allAnomaliesUpperTierThree = getAnomalies2(series, window, upperBoundTierThree, lowerBoundTierThree,
                                                       upperBoundYOY=upperBoundYOYTierThree, lowerBoundYOY=lowerBoundYOYTierThree, lbound=False, isYOY=yoyTest)
            allAnomaliesLowerTierThree = getAnomalies2(series, window, upperBoundTierThree, lowerBoundTierThree,
                                                       upperBoundYOY=upperBoundYOYTierThree, lowerBoundYOY=lowerBoundYOYTierThree, lbound=True, isYOY=yoyTest)

            # Create temp output DF
            finaldf = series.copy()
            del series
            #print("series copy: ", finaldf)

            # Add Current Year Bounds to output df
            finaldf = finaldf.merge(
            	upperBoundTierOne, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	upperBoundTierTwo, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	upperBoundTierThree, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	lowerBoundTierOne, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	lowerBoundTierTwo, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	lowerBoundTierThree, left_index=True, right_index=True)
            #print("add bounds: ", finaldf)
            del upperBoundTierOne, upperBoundTierTwo, upperBoundTierThree, lowerBoundTierOne, lowerBoundTierTwo, lowerBoundTierThree

            # Add YOY Bounds to output df
            finaldf = finaldf.merge(
            	upperBoundYOYTierOne, left_index=True, right_index=True, how="left")
            finaldf = finaldf.merge(
            	upperBoundYOYTierTwo, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	upperBoundYOYTierThree, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	lowerBoundYOYTierOne, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	lowerBoundYOYTierTwo, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	lowerBoundYOYTierThree, left_index=True, right_index=True)
            del upperBoundYOYTierOne, upperBoundYOYTierTwo, upperBoundYOYTierThree, lowerBoundYOYTierOne, lowerBoundYOYTierTwo, lowerBoundYOYTierThree

            # Drop extra anomaly column that I honestly don't know what it exists for
            dropCol = allAnomaliesUpperTierOne.columns.values.tolist()[-1]
            allAnomaliesUpperTierOne = allAnomaliesUpperTierOne.drop([dropCol],axis=1)
            allAnomaliesUpperTierTwo = allAnomaliesUpperTierTwo.drop([dropCol],axis=1)
            allAnomaliesUpperTierThree = allAnomaliesUpperTierThree.drop([dropCol],axis=1)
            allAnomaliesLowerTierOne = allAnomaliesLowerTierOne.drop([dropCol],axis=1)
            allAnomaliesLowerTierTwo = allAnomaliesLowerTierTwo.drop([dropCol],axis=1)
            allAnomaliesLowerTierThree = allAnomaliesLowerTierThree.drop([dropCol],axis=1)
            #print("check anomalies tier one upper: ", allAnomaliesUpperTierOne)

            # Merge in the anomalies
            finaldf = finaldf.merge(
            	allAnomaliesUpperTierOne, left_index=True, right_index=True, how="left")
            finaldf = finaldf.merge(
            	allAnomaliesUpperTierTwo, left_index=True, right_index=True, how="left")
            finaldf = finaldf.merge(
            	allAnomaliesUpperTierThree, left_index=True, right_index=True, how="left")
            finaldf = finaldf.merge(
            	allAnomaliesLowerTierOne, left_index=True, right_index=True, how="left")
            finaldf = finaldf.merge(
            	allAnomaliesLowerTierTwo, left_index=True, right_index=True, how="left")
            finaldf = finaldf.merge(
            	allAnomaliesLowerTierThree, left_index=True, right_index=True, how="left")
            #print("add anomalies: ", finaldf)
            del allAnomaliesUpperTierOne, allAnomaliesUpperTierTwo, allAnomaliesUpperTierThree, allAnomaliesLowerTierOne, allAnomaliesLowerTierTwo, allAnomaliesLowerTierThree

            # Rename Columns
            finaldf.columns = ['actuals',
                               'upperBoundTierOne', 'upperBoundTierTwo', 'upperBoundTierThree',
                               'lowerBoundTierOne', 'lowerBoundTierTwo', 'lowerBoundTierThree',
                               'upperBoundYOYTierOne', 'upperBoundYOYTierTwo', 'upperBoundYOYTierThree',
                               'lowerBoundYOYTierOne', 'lowerBoundYOYTierTwo', 'lowerBoundYOYTierThree',
                               'allAnomaliesUpperTierOne', 'allAnomaliesUpperTierTwo', 'allAnomaliesUpperTierThree', 'allAnomaliesLowerTierOne', 'allAnomaliesLowerTierTwo', 'allAnomaliesLowerTierThree']

            finaldf[dimColumnOne] = category

            finaldf.reset_index()

            print("Shape of dataframe: ", finaldf.shape)

            tempFrames.append(finaldf)

        else:
            print("Category won't run, moving on", category)
    #print("TEMP FRAMES SHAPE: ", tempFrames)
    outputDF = pd.concat(tempFrames)
    print('\n7. Table finished populating - saving')

    return outputDF


# Run the Handler

In [0]:
handler(dfbq,colToClean,dateColumn,dimColumnOne,dimColumnTwo,metricColumn)

1. BigQuery Table Collected

2. Data Frame Created

4. Filtered data to top  100 landingPage

5. Created Data ranges

6. Created current and previous year tables
['www.cbtnuggets.com/learn', 'www.cbtnuggets.com/search', '(not set)', 'www.cbtnuggets.com/login', 'www.cbtnuggets.com/']

6. Begin looping through dimension
running function:  www.cbtnuggets.com/learn



Anomalies by landingPage: www.cbtnuggets.com/learn 

Shape of dataframe:  (30, 20)
running function:  www.cbtnuggets.com/search



Anomalies by landingPage: www.cbtnuggets.com/search 

Shape of dataframe:  (30, 20)
running function:  (not set)



Anomalies by landingPage: (not set) 

Shape of dataframe:  (30, 20)
running function:  www.cbtnuggets.com/login



Anomalies by landingPage: www.cbtnuggets.com/login 

Shape of dataframe:  (30, 20)
running function:  www.cbtnuggets.com/



Anomalies by landingPage: www.cbtnuggets.com/ 

Shape of dataframe:  (30, 20)

7. Table finished populating - saving


,actuals,upperBoundTierOne,upperBoundTierTwo,upperBoundTierThree,lowerBoundTierOne,lowerBoundTierTwo,lowerBoundTierThree,upperBoundYOYTierOne,upperBoundYOYTierTwo,upperBoundYOYTierThree,lowerBoundYOYTierOne,lowerBoundYOYTierTwo,lowerBoundYOYTierThree,allAnomaliesUpperTierOne,allAnomaliesUpperTierTwo,allAnomaliesUpperTierThree,allAnomaliesLowerTierOne,allAnomaliesLowerTierTwo,allAnomaliesLowerTierThree,landingPage
myDate,,,,,,,,,,,,,,,,,,,,
2019-12-18,2454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,www.cbtnuggets.com/learn
2019-12-19,2282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,www.cbtnuggets.com/learn
2019-12-20,1789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,www.cbtnuggets.com/learn
2019-12-21,1108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,www.cbtnuggets.com/learn
2019-12-22,1304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,www.cbtnuggets.com/learn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-12,1373,2456.256181,2798.086227,3139.916272,1142.243819,800.413773,458.583728,5484.539896,6463.108201,7441.676507,1831.210104,852.641799,-125.926507,NaN,NaN,NaN,NaN,NaN,NaN,www.cbtnuggets.com/
2020-01-13,2358,2598.381181,2940.211227,3282.041272,1284.368819,942.538773,600.708728,5517.664896,6496.233201,7474.801507,1864.335104,885.766799,-92.801507,NaN,NaN,NaN,NaN,NaN,NaN,www.cbtnuggets.com/
2020-01-14,2380,2610.381181,2952.211227,3294.041272,1296.368819,954.538773,612.708728,5790.289896,6768.858201,7747.426507,2136.960104,1158.391799,179.823493,NaN,NaN,NaN,NaN,NaN,NaN,www.cbtnuggets.com/


# SQL Handler - Do Not Run

In [0]:
def handlerwSQL(toClean=True, lpCount=100, series_length=365, window=8):

    #Generate Query & initial table
    client = bq.Client()
    dfbq = client.query(sql, project=id).to_dataframe()
    dfCopy = dfbq.copy()
    del dfbq
    print("1. BigQuery Table Collected")

    # Create Initial DataFrame - Cleaning Query Strings if Applicable
    #def prepSingleDataFrame(df, clean, cleanCol, dateCol, metric, *dims):
    df = prepSingleDataFrame(dfCopy, toClean, colToClean,
                             dateColumn, metricColumn, dimColumnOne, dimColumnTwo)
    print("\n2. Data Frame Created")

    # Get Filter list for primary dimension
    #focusList = focus(df, dimColumnOne, metricColumn, n=lpCount)
    #print("\n3. Focus list created of top ", lpCount, dimColumnOne)
    

    # Update table to only include focusList items
    dfFiltered = filterDim(focusList, df, dimColumnOne)
    print("\n4. Filtered data to top ", lpCount, dimColumnOne)
    del df

    # Set Dates
    end_date = pd.to_datetime(
    	(datetime.now() - timedelta(1)).strftime('%Y-%m-%d')).date()
    start_date = pd.to_datetime(
    	(datetime.now() - timedelta(30)).strftime('%Y-%m-%d')).date()
    start_dateYOY = pd.to_datetime(
    	(datetime.now() - timedelta(396)).strftime('%Y-%m-%d')).date()  # test at 396,365
    end_dateYOY = pd.to_datetime(
    	(datetime.now() - timedelta(366)).strftime('%Y-%m-%d')).date()
    print("\n5. Created Data ranges")

    # Create Current Year Table and Previous Year Table
    dfNow, dfThen = nowAndLater(dfFiltered, dateColumn, metricColumn,
                                start_date, end_date, start_dateYOY, end_dateYOY, dimColumnOne)
    print("\n6. Created current and previous year tables")
    del end_date, start_date, start_dateYOY, end_dateYOY

    # Create Columns Item
    cols = dfThen.columns

    # Create Categories
    dfCategories = intersection(dfThen.index.get_level_values(
    	1).unique(), dfNow.index.get_level_values(1).unique())

    # Create Category Counter - Set to 0
    categoryCounter = 0  # IS THIS NECESSARY?=['/]

    # Create Initial Data Frame
    tempFrames = []

    # Error Handling, print my list of categories
    print(dfCategories)
    print('\n6. Begin looping through dimension')

    # Loop Through
    for category in dfCategories:
        print("running function: ", category)

        # Current Year Data
        dfTemp = dfNow.loc(axis=0)[pd.IndexSlice[:, category]].copy()
        dfTempYOY = dfThen.loc(
            axis=0)[pd.IndexSlice[:, category]].copy()   # YOY Data

        # Current Year Data
        dfTemp.reset_index(level=[1], inplace=True)
        # YOY Data
        dfTempYOY.reset_index(level=[1], inplace=True)

        print("\n\n\nAnomalies by " +
              dfTemp.columns[0] + ": " + category + ' \n')
        # Current Year Data
        dfTemp.drop(dfTemp.columns[0], axis=1, inplace=True)
        # YOY Data
        dfTempYOY.drop(dfTempYOY.columns[0], axis=1, inplace=True)
        # Shift dates forward one day
        dfTempYOY.index = dfTempYOY.index.shift(0, freq='D')
        dfTempYOY = dfTempYOY.tail(30)
        dfTemp = dfTemp.asfreq('D').fillna(0)  # FillNA with O
        dfTempYOY = dfTempYOY.asfreq('D').fillna(0)  # FillNA with O

        # Assign Series
        series = dfTemp
        seriesYOY = dfTempYOY
        #print(series)
        #print(seriesYOY)

        if len(series) > window:

            # Set series
            series = series.tail(series_length)

            # Get rolling mean for current year
            rolling_mean = series.rolling(window=window).mean()

            # create Len of Series Test values
            seriesLen = len(series)
            seriesLenYOY = len(seriesYOY)

            #shift index forward 1 year to overlay last year's data over current year's
            seriesYOY.index = seriesYOY.index.shift(365, freq='D')
            seriesYOY = seriesYOY.tail(series_length)

            # Create YOY Rolling Mean
            rolling_meanYOY = seriesYOY.rolling(window=window).mean()

            # Create yoyTest Variable
            if seriesLen == seriesLenYOY:  # Both Series are Equal, run current and YOY
                yoyTest = True
            elif seriesLen > seriesLenYOY:
                yoyTest = False
            elif seriesLen != seriesLenYOY:
                yoyTest = False

            # Create Bounds
            upperBoundTierOne = getBounds(
            	series, window, rolling_mean, scale=1, lowerBound=False)
            lowerBoundTierOne = getBounds(
            	series, window, rolling_mean, scale=1, lowerBound=True)

            upperBoundTierTwo = getBounds(
            	series, window, rolling_mean, scale=2, lowerBound=False)
            lowerBoundTierTwo = getBounds(
            	series, window, rolling_mean, scale=2, lowerBound=True)

            upperBoundTierThree = getBounds(
            	series, window, rolling_mean, scale=3, lowerBound=False)
            lowerBoundTierThree = getBounds(
            	series, window, rolling_mean, scale=3, lowerBound=True)

            # Create YOY Bounds
            if yoyTest == True:  # Create YOY if YOY Is True
                upperBoundYOYTierOne = getBounds(
                    seriesYOY, window, rolling_meanYOY, scale=1, lowerBound=False)
                lowerBoundYOYTierOne = getBounds(
                    seriesYOY, window, rolling_meanYOY, scale=1, lowerBound=True)
                upperBoundYOYTierTwo = getBounds(
                    seriesYOY, window, rolling_meanYOY, scale=2, lowerBound=False)
                lowerBoundYOYTierTwo = getBounds(
                    seriesYOY, window, rolling_meanYOY, scale=2, lowerBound=True)
                upperBoundYOYTierThree = getBounds(
                    seriesYOY, window, rolling_meanYOY, scale=3, lowerBound=False)
                lowerBoundYOYTierThree = getBounds(
                    seriesYOY, window, rolling_meanYOY, scale=3, lowerBound=True)
            else:
                replaceVal = series.copy()
                #print("Copy: ", replaceVal)
                replaceVal[metricColumn] = np.nan
                #print(replaceVal)
                upperBoundYOYTierOne = replaceVal
                lowerBoundYOYTierOne = replaceVal
                upperBoundYOYTierTwo = replaceVal
                lowerBoundYOYTierTwo = replaceVal
                upperBoundYOYTierThree = replaceVal
                lowerBoundYOYTierThree = replaceVal

            #print(series['sessions'])
            #print(upperBoundYOYTierOne)

            # Find Anomalies
            # Tier One
            allAnomaliesUpperTierOne = getAnomalies2(series, window, upperBoundTierOne, lowerBoundTierOne,
                                                     upperBoundYOY=upperBoundYOYTierOne, lowerBoundYOY=lowerBoundYOYTierOne, lbound=False, isYOY=yoyTest)
            allAnomaliesLowerTierOne = getAnomalies2(series, window, upperBoundTierOne, lowerBoundTierOne,
                                                     upperBoundYOY=upperBoundYOYTierOne, lowerBoundYOY=lowerBoundYOYTierOne, lbound=True, isYOY=yoyTest)

            # Tier Two
            allAnomaliesUpperTierTwo = getAnomalies2(series, window, upperBoundTierTwo, lowerBoundTierTwo,
                                                     upperBoundYOY=upperBoundYOYTierTwo, lowerBoundYOY=lowerBoundYOYTierTwo, lbound=False, isYOY=yoyTest)
            allAnomaliesLowerTierTwo = getAnomalies2(series, window, upperBoundTierTwo, lowerBoundTierTwo,
                                                     upperBoundYOY=upperBoundYOYTierTwo, lowerBoundYOY=lowerBoundYOYTierTwo, lbound=True, isYOY=yoyTest)

            # Tier Three
            allAnomaliesUpperTierThree = getAnomalies2(series, window, upperBoundTierThree, lowerBoundTierThree,
                                                       upperBoundYOY=upperBoundYOYTierThree, lowerBoundYOY=lowerBoundYOYTierThree, lbound=False, isYOY=yoyTest)
            allAnomaliesLowerTierThree = getAnomalies2(series, window, upperBoundTierThree, lowerBoundTierThree,
                                                       upperBoundYOY=upperBoundYOYTierThree, lowerBoundYOY=lowerBoundYOYTierThree, lbound=True, isYOY=yoyTest)

            # Create temp output DF
            finaldf = series.copy()
            del series
            #print("series copy: ", finaldf)

            # Add Current Year Bounds to output df
            finaldf = finaldf.merge(
            	upperBoundTierOne, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	upperBoundTierTwo, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	upperBoundTierThree, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	lowerBoundTierOne, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	lowerBoundTierTwo, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	lowerBoundTierThree, left_index=True, right_index=True)
            #print("add bounds: ", finaldf)
            del upperBoundTierOne, upperBoundTierTwo, upperBoundTierThree, lowerBoundTierOne, lowerBoundTierTwo, lowerBoundTierThree

            # Add YOY Bounds to output df
            finaldf = finaldf.merge(
            	upperBoundYOYTierOne, left_index=True, right_index=True, how="left")
            finaldf = finaldf.merge(
            	upperBoundYOYTierTwo, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	upperBoundYOYTierThree, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	lowerBoundYOYTierOne, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	lowerBoundYOYTierTwo, left_index=True, right_index=True)
            finaldf = finaldf.merge(
            	lowerBoundYOYTierThree, left_index=True, right_index=True)
            del upperBoundYOYTierOne, upperBoundYOYTierTwo, upperBoundYOYTierThree, lowerBoundYOYTierOne, lowerBoundYOYTierTwo, lowerBoundYOYTierThree

            # Drop extra anomaly column that I honestly don't know what it exists for
            dropCol = allAnomaliesUpperTierOne.columns.values.tolist()[-1]
            allAnomaliesUpperTierOne = allAnomaliesUpperTierOne.drop([dropCol],axis=1)
            allAnomaliesUpperTierTwo = allAnomaliesUpperTierTwo.drop([dropCol],axis=1)
            allAnomaliesUpperTierThree = allAnomaliesUpperTierThree.drop([dropCol],axis=1)
            allAnomaliesLowerTierOne = allAnomaliesLowerTierOne.drop([dropCol],axis=1)
            allAnomaliesLowerTierTwo = allAnomaliesLowerTierTwo.drop([dropCol],axis=1)
            allAnomaliesLowerTierThree = allAnomaliesLowerTierThree.drop([dropCol],axis=1)
            #print("check anomalies tier one upper: ", allAnomaliesUpperTierOne)

            # Merge in the anomalies
            finaldf = finaldf.merge(
            	allAnomaliesUpperTierOne, left_index=True, right_index=True, how="left")
            finaldf = finaldf.merge(
            	allAnomaliesUpperTierTwo, left_index=True, right_index=True, how="left")
            finaldf = finaldf.merge(
            	allAnomaliesUpperTierThree, left_index=True, right_index=True, how="left")
            finaldf = finaldf.merge(
            	allAnomaliesLowerTierOne, left_index=True, right_index=True, how="left")
            finaldf = finaldf.merge(
            	allAnomaliesLowerTierTwo, left_index=True, right_index=True, how="left")
            finaldf = finaldf.merge(
            	allAnomaliesLowerTierThree, left_index=True, right_index=True, how="left")
            #print("add anomalies: ", finaldf)
            del allAnomaliesUpperTierOne, allAnomaliesUpperTierTwo, allAnomaliesUpperTierThree, allAnomaliesLowerTierOne, allAnomaliesLowerTierTwo, allAnomaliesLowerTierThree

            # Rename Columns
            finaldf.columns = ['actuals',
                               'upperBoundTierOne', 'upperBoundTierTwo', 'upperBoundTierThree',
                               'lowerBoundTierOne', 'lowerBoundTierTwo', 'lowerBoundTierThree',
                               'upperBoundYOYTierOne', 'upperBoundYOYTierTwo', 'upperBoundYOYTierThree',
                               'lowerBoundYOYTierOne', 'lowerBoundYOYTierTwo', 'lowerBoundYOYTierThree',
                               'allAnomaliesUpperTierOne', 'allAnomaliesUpperTierTwo', 'allAnomaliesUpperTierThree', 'allAnomaliesLowerTierOne', 'allAnomaliesLowerTierTwo', 'allAnomaliesLowerTierThree']

            finaldf[dimColumnOne] = category

            finaldf.reset_index()

            print("Shape of dataframe: ", finaldf.shape)

            tempFrames.append(finaldf)

        else:
            print("Category won't run, moving on", category)
    #print("TEMP FRAMES SHAPE: ", tempFrames)
    outputDF = pd.concat(tempFrames)
    print('\n7. Table finished populating - writing to BQ')

    ## WRITE TO BQ ##
    # Since string columns use the "object" dtype, pass in a (partial) schema
    # to ensure the correct BigQuery data type.
    job_config = bigquery.LoadJobConfig(schema=[
        bigquery.SchemaField("myDate", "DATE"),
        bigquery.SchemaField("actuals", "FLOAT"),
        bigquery.SchemaField("upperBoundTierOne", "FLOAT"),
        bigquery.SchemaField("upperBoundTierTwo", "FLOAT"),
        bigquery.SchemaField("upperBoundTierThree", "FLOAT"),
        bigquery.SchemaField("lowerBoundTierOne", "FLOAT"),
        bigquery.SchemaField("lowerBoundTierTwo", "FLOAT"),
        bigquery.SchemaField("lowerBoundTierThree", "FLOAT"),
        bigquery.SchemaField("upperBoundYOYTierOne", "FLOAT"),
        bigquery.SchemaField("upperBoundYOYTierTwo", "FLOAT"),
        bigquery.SchemaField("upperBoundYOYTierThree", "FLOAT"),
        bigquery.SchemaField("lowerBoundYOYTierOne", "FLOAT"),
        bigquery.SchemaField("lowerBoundYOYTierTwo", "FLOAT"),
        bigquery.SchemaField("lowerBoundYOYTierThree", "FLOAT"),
        bigquery.SchemaField("allAnomaliesUpperTierOne", "FLOAT"),
        bigquery.SchemaField("allAnomaliesUpperTierTwo", "FLOAT"),
        bigquery.SchemaField("allAnomaliesUpperTierThree", "FLOAT"),
        bigquery.SchemaField("allAnomaliesLowerTierOne", "FLOAT"),
        bigquery.SchemaField("allAnomaliesLowerTierTwo", "FLOAT"),
        bigquery.SchemaField("allAnomaliesLowerTierThree", "FLOAT"),
        bigquery.SchemaField(dimColumnOne, "STRING")
    ],
        write_disposition="WRITE_TRUNCATE")

    job = client.load_table_from_dataframe(
        outputDF, finalTable, job_config=job_config
    )

    # Wait for the load job to complete.
    job.result()
    print('\n8. Written to BQ')

    return outputDF
